In [ ]:
settings = {
    "xsrf_cookies": False,
}

In [ ]:
import pandas as pd
import itertools
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import matplotlib.pyplot as plt
import numpy as np

#load data labels and RDkit fingerprints
#using pickled data
with open('../dumps/combined_dataset.pkl', 'rb') as f:
      data = pickle.load(f)

N=len(data.loc[data['target'] == 'TMPRSS2']) #number of compounds tested against TMPRSS2 itself
n_test=round(N/3) #withold for testing
fp_dark=np.load('../dumps/DarkChemicalMatter_morgan_fingerprints.npz')
fp_dark=fp_dark['fps']


In [ ]:
#convert to array
#using merged data from pickle
datalabels=list(data['target'])
fps_merged=np.stack(data['morgan_fingerprint'])
ac_merged_scaled=np.asarray(data['acvalue_scaled_to_tmprss2'])
cids=list(data['cid'])
act_list=list(data['activity_target'])
activity_scaled=[]
activity=[]
for i in range(len(act_list)):
    if act_list[i] == 'Active':
        activity.append(1)
    else:
        activity.append(0)
    if ac_merged_scaled[i]>0:
        if act_list[i] == 'Active':
            activity_scaled.append(1)
        else:
            activity_scaled.append(0)
#alternatively
ac_merged=np.asarray(data['acvalue_target'])
#keep only the positive vals in ac_merged
fps_merged_scaled=fps_merged[ac_merged_scaled>0]
cids_scaled=[cid for i, cid in enumerate(cids) if ac_merged_scaled[i]>0]
ac_merged_scaled=ac_merged[ac_merged_scaled>0]


activity=np.array(activity)
ac_merged=np.array(ac_merged)
ac_merged=-np.log10(ac_merged)
activity_scaled=np.array(activity_scaled)
ac_merged_scaled=np.array(ac_merged_scaled)
ac_merged_scaled=-np.log10(ac_merged_scaled)

In [ ]:
#split train and test 
#half of tmprss2 active compounds to each set
import random
s=np.arange(N)
random.shuffle(s)
cut=n_test
test=s[0:cut]
train=s[cut::]

#add dark data. here we generate 1 test set that includes only data from TMPRSS2 dataset + negative examples
#and training data culled from all protein datasets, + negative examples that are not in the test set.
#the samples from TMPRSS2, and negative examples are sampled randomly to be about 50 percent.
s=np.arange(len(fp_dark))
random.shuffle(s)
cut=round(len(fp_dark)/2)
test_dark=s[0:cut]
train_dark=s[cut::]
X_test=np.concatenate((fps_merged_scaled[test], fp_dark[test_dark]))
X_train=np.concatenate((fps_merged_scaled[train], fps_merged_scaled[92::], fp_dark[train_dark]))
y_train=np.concatenate((activity_scaled[train], activity_scaled[92::], np.zeros([len(train_dark)])))
y2_train=np.concatenate((ac_merged_scaled[train], ac_merged_scaled[92::], np.zeros([len(train_dark)])))
y_test=np.concatenate((activity_scaled[test], np.zeros([len(test_dark)])))
y2_test=np.concatenate((ac_merged_scaled[test],np.zeros([len(test_dark)])))

X_test_u=np.concatenate((fps_merged[test], fp_dark[test_dark]))
X_train_u=np.concatenate((fps_merged[train], fps_merged[92::], fp_dark[train_dark]))
y_train_u=np.concatenate((activity[train], activity[92::], np.zeros([len(train_dark)])))
y2_train_u=np.concatenate((ac_merged[train], ac_merged[92::], np.zeros([len(train_dark)])))
y_test_u=np.concatenate((activity[test], np.zeros([len(test_dark)])))
y2_test_u=np.concatenate((ac_merged[test],np.zeros([len(test_dark)])))

In [ ]:
#generate big training set
X_train_all=np.concatenate((fps_merged_scaled, fp_dark))
y2_train_all=np.concatenate((ac_merged_scaled, np.zeros([len(fp_dark)])))
y_train_all=np.concatenate((activity_scaled, np.zeros([len(fp_dark)])))

In [ ]:
###YOU CAN SKIP THIS STEP IF NOT RUNNING THE CLASSIFIER###
#generate RF: classifier based on whether or not compound is thought to be active.
#split training and test set, keep random_state to be an integer for reproducibility
#train the forest (this can take a while)
#in the example, about 2/3 go to training and 1/3 go to test
#X_train, X_test, y_train, y_test = train_test_split(fps_merged,activity, test_size=0.33, random_state=39)
rf=RandomForestClassifier(verbose=2, n_estimators=100, random_state=111)
rf.fit(np.asarray(X_train), np.asarray(y_train))

In [ ]:
y_pred=rf.predict(X_test)
print(rf.score(X_test, y_test))

In [ ]:
#get the index of the features (of RDkit fingerprint) that were important
importances = rf.feature_importances_
featurenums = np.array([str(x).zfill(2) for x in range(len(importances))])
indices = np.argsort(importances)[::-1][0:25]#get the 25 most important features
plt.title('Feature Importances (train set)')
plt.bar(range(len(indices)), importances[indices], align='center')
plt.ylabel('Relative Importance')
plt.xticks(range(len(indices)), featurenums[indices], rotation=90)
plt.show()
#output metrics
print(classification_report(y_test, y_pred))


In [ ]:
#REGRESSION
#testing, but without the negative training examples.
#take the same random vector of subset of true TMPSSR2 values as test set, and everything else i.e.
# the ac values from assay against a different membrane protein is combined into training set
#drop nan values
X_test_reg=fps_merged[test]
X_train_reg=np.concatenate((fps_merged[train], fps_merged[92::]))
y2_train_reg=np.concatenate((ac_merged[train], ac_merged[92::]))
y2_test_reg=ac_merged[test]
nan_array=np.isnan(y2_train_reg)
X_train_reg=X_train_reg[~nan_array]
y2_train_reg=y2_train_reg[~nan_array]
#and generate for scaled arrays
X_train_reg_scaled=np.concatenate((fps_merged_scaled[train], fps_merged_scaled[92::]))
y2_train_reg_scaled=np.concatenate((ac_merged_scaled[train], ac_merged_scaled[92::]))

#here we have trained 2 regressors
#rg_pos_only <- no negative training examples and unscaled data
#rg_pos_only_scaled <- no negative training examples and scaled data
rg_pos_only=RandomForestRegressor(verbose=2, n_estimators=50, random_state=111)
rg_pos_only_scaled=RandomForestRegressor(verbose=2, n_estimators=50, random_state=111)
rg_pos_only.fit(np.asarray(X_train_reg), np.asarray(y2_train_reg))
rg_pos_only_scaled.fit(np.asarray(X_train_reg_scaled), np.asarray(y2_train_reg_scaled))

In [ ]:
from sklearn.metrics import r2_score
#the test set only contains unscaled values from TMPRSS2 assay so its the same
#scaled data, regression on positives
y_pred_regr_pos=rg_pos_only.predict(X_test_reg)
print(r2_score(y2_test_reg, y_pred_regr_pos))

#unscaled data, regression on positives
y_pred_regr_scaled_pos=rg_pos_only_scaled.predict(X_test_reg)
print(r2_score(y2_test_reg, y_pred_regr_scaled_pos))

#not great, but slightly better score for "scaled" dataset.

In [ ]:
#bloc for some hyperparameter tuning
#to add stuff later
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rg_scaled.get_params())

In [ ]:
importances = rg_pos_only_scaled.feature_importances_
featurenums = np.array([str(x).zfill(2) for x in range(len(importances))])
indices = np.argsort(importances)[::-1][0:25]#get the 25 most important features
plt.title('Feature Importances (train set)')
plt.bar(range(len(indices)), importances[indices], align='center')
plt.ylabel('Relative Importance')
plt.xticks(range(len(indices)), featurenums[indices], rotation=90)
plt.show()

In [ ]:
#plots, first is for unscaled regression
plt.plot(y2_test_reg, 'b.', label = 'actual')
# Plot the predicted values
plt.plot(y_pred_regr_pos, 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 

In [ ]:
#second is for scaled regression
plt.plot(y2_test_reg_scaled, 'b.', label = 'actual')
# Plot the predicted values
plt.plot(y_pred_regr_scaled_pos, 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 
#note does a slightly better job at predicting the low ac-val which is what we care about?
#mumber 13 has low ac value and is overpredicted by model always.

In [ ]:
#now fit everything
#no negative training examples
#all training data from 
rg_all=RandomForestRegressor(verbose=2, n_estimators=100, random_state=111)
rg_all.fit(np.asarray(X_train_all), np.asarray(y2_train_all))

In [ ]:
reframe=np.load('../dumps/reframe_fp.npz')
molnames=reframe['arr_1']
reframe=reframe['arr_0']

screening=np.load('../dumps/screening_fp.npz')
molnames_all=screening['arr_1']
screening=screening['arr_0']

In [ ]:
rg_all=RandomForestRegressor(verbose=2, n_estimators=100, random_state=111)
rg_all.fit(np.asarray(fps_merged_scaled), np.asarray(ac_merged_scaled)) #use scaled bc it performs slightly better in the validation case

In [ ]:
# Open the file to save as pkl file
rf_pkl_filename = 'rf_reg_20200616.pkl'
rf_model_pkl = open(rf_pkl_filename, 'wb')
pickle.dump(rg_all2, rf_model_pkl)
# Close the pickle instances
rf_model_pkl.close()

In [ ]:
predicted_activity=rg_all.predict(screening)

In [ ]:
#plot and print the top 20 hits
#plotted values are in -log10 form (higher is better)
plt.plot(predicted_activity, 'ro', label = 'prediction')
indices=np.argpartition(predicted_activity, -20)[-20:]
print(molnames_all[indices][np.argsort(predicted_activity[indices])][::-1])
print(np.sort(predicted_activity[indices])[::-1])

In [ ]:
#old code snippet for running unscaled values
nan_array=np.isnan(ac_merged)
fps_merged=fps_merged[~nan_array]
ac_merged=ac_merged[~nan_array]
rg_all3=RandomForestRegressor(verbose=2, n_estimators=100, random_state=111)
rg_all3.fit(np.asarray(fps_merged), np.asarray(ac_merged))